In [1]:
from lxml import html
from requests import get
from pprint import pprint
from pymongo import MongoClient
import hashlib
from datetime import datetime

In [2]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'
header = {
        'User-Agent':user_agent
    }

# Собираем парсеры

In [260]:
# Парсер яндекса
def news_yandex():
    response = get('https://yandex.ru/news/',headers = header)
    root = html.fromstring(response.text)
    items_tables = root.xpath("//table//td")
    result = []
    #hot news block
    info = {}
    info['link'] ='https://yandex.ru/'+  root.xpath("/html[1]/body[1]/div[3]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/h2[1]/a[1]/@href")[0].split('?')[0]
    info['name'] = root.xpath("/html[1]/body[1]/div[3]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/h2[1]/a[1]//text()")[0]
    info['source'],info['time_in_news'] = root.xpath("/html[1]/body[1]/div[3]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/text()")[0].rsplit(' ',1)
    info['date_scrapped'] = datetime.date(datetime.now()).strftime('%d/%m/%Y')
    info['_id'] = hashlib.md5(info['link'].encode()).hexdigest()
    result.append(info)
    for item in items_tables:
        info = {}
        info['link'] = 'https://yandex.ru/' + item.xpath(".//h2[contains(@class, 'story__title')]//@href")[0].split('?')[0]
        info['name'] = item.xpath(".//h2[contains(@class, 'story__title')]//text()")
        info['source'],info['time_in_news'] = item.xpath(".//div[contains(@class, 'story__info')]//text()")[0].rsplit(' ',1)
        info['date_scrapped'] = datetime.date(datetime.now()).strftime('%d/%m/%Y')
        info['_id'] = hashlib.md5(info['link'].encode()).hexdigest()
        result.append(info)
    return result

In [199]:
# Парсер мейл.ру

In [200]:
#Провлаиваемся на страницу новости и забираем от тда источник и время
def mail_Parse_source(link):
    response = get(link, headers = header)
    root = html.fromstring(response.text)
    source = root.xpath("//a[@class = 'link color_gray breadcrumbs__link']//text()")[0]
    time_in_news = root.xpath("//span[contains(@class,'note')]//text()")[0].split(' ')[0]
    return source, time_in_news

In [266]:
# на мейо ру соишком много новостей. Я решилбрать первые 5, с картинками
def news_Mail():
    response = get('https://news.mail.ru/', headers = header)
    root = html.fromstring(response.text)
    day_news_main = root.xpath("//td[@class='daynews__main']")

    result = []
    for item in day_news_main:
        info = {}
        info['link'] = 'https://news.mail.ru' + item.xpath(".//@href")[0]
        info['name'] = item.xpath(".//span[contains(@class, 'photo__captions')]//text()")[0]
        info['source'],info['time_in_news'] = mail_Parse_source(link)
        info['date_scrapped'] = datetime.date(datetime.now()).strftime('%d/%m/%Y')
        info['_id'] = hashlib.md5(info['link'].encode()).hexdigest()
        result.append(info)
    for item in day_news_item:
        info = {}
        info['link'] = 'https://news.mail.ru' + item.xpath(".//@href")[0]
        info['name'] = item.xpath(".//span[contains(@class, 'photo__captions')]//text()")[0]
        info['source'],info['time_in_news'] = mail_Parse_source(link)
        info['date_scrapped'] = datetime.date(datetime.now()).strftime('%d/%m/%Y')
        info['_id'] = hashlib.md5(info['link'].encode()).hexdigest()
        result.append(info)
    return result

In [202]:
def lenta_source(link):
    response = get(link, headers = header)
    root = html.fromstring(response.text)
    time_in_news = root.xpath('//div [@class = "b-topic__content"]//time/text()')[0].split(',')[0].lstrip(' ')
    return time_in_news

In [203]:
#Лента ру
def news_Lenta():
    response = get('https://lenta.ru', headers = header)
    root = html.fromstring(response.text)
    main_news = root.xpath("//div[@class='b-yellow-box__wrap']/div[@class = 'item']")
    for item in main_news:
        info = {}
        if item.xpath(".//@href")[0][0]=='/':
            info['link'] = 'https://lenta.ru' + item.xpath(".//@href")[0]
        else:
            info['link'] = item.xpath(".//@href")[0]
        info['name'] = item.xpath(".//text()")[0]
        info['source'] = "Lenta"
        try:
            info['time_in_news'] = lenta_source(info['link'])
        except:
            info['time_in_news']= None
        info['date_scrapped'] = datetime.date(datetime.now()).strftime('%d/%m/%Y')
        info['_id'] = hashlib.md5(info['link'].encode()).hexdigest()
        result.append(info)
    return result

# Загружаем все в базу

In [204]:
client = MongoClient('127.0.0.1',27017)

In [205]:
db = client['News']
news = db.news


In [207]:
#create func that will get data from all sources
def getData():
    data = news_yandex()
    data+=news_Mail()
    data+=news_Lenta()
    return data

In [269]:
def addData_All(data,db):
    count = 0
    count2 = 0
    for e in data:
        try:
            db.insert_one(e)
            count2+=1
        except:
            count+=1
            continue
    print('Done', count,': Duplicates dropped','\n', count2, 'new values added')

In [268]:
#getData()

In [270]:
addData_All(getData(),news)

Done 18 : Duplicates dropped 
 81 new values added


In [276]:
for new in news.find({}).sort('source').limit(3):
    pprint(new)

{'_id': '5aeaa99f6c87c221f3bb444661ff742a',
 'date_scrapped': '22/04/2020',
 'link': 'https://yandex.ru//news/story/Postavki_iPhone_grozyat_snizitsya_na_chetvert_v_tekushhem_kvartale--768e7fdf18bbba1b465d658e3569b5fb',
 'name': ['Поставки iPhone грозят снизиться на четверть в текущем квартале'],
 'source': '3DNews',
 'time_in_news': '12:05'}
{'_id': '9e1ef9a270f5717bf7cd7e996fef9cee',
 'date_scrapped': '22/04/2020',
 'link': 'https://yandex.ru//news/story/AMD_predstavila_chipy_Ryzen_3_3100_i_Ryzen_3_3300X--e5b9e594bdf96a1112ed8a81c1cd018a',
 'name': ['AMD представила чипы Ryzen 3 3100 и Ryzen 3 3300X'],
 'source': 'Astera',
 'time_in_news': '10:26'}
{'_id': '57460601b8058a29a373c580fde3c384',
 'date_scrapped': '22/04/2020',
 'link': 'https://yandex.ru//news/story/Sebastyan_Fettel_otkazalsya_ot_predlozheniya_Ferrari_po_novomu_kontraktu--1ef2dbac024a065bdd6251350aedac0f',
 'name': ['Себастьян Феттель отказался от предложения Ferrari по новому '
          'контракту'],
 'source': 'Autospo